In [1]:
import pandas as pd
import numpy as np 
import datetime
import os

def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)
            
start_date_2019Q4=datetime.date(2019,11,3)
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2)


In [2]:
# POS date
pos_list=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
pos_list=[x for x in pos_list if "daily" in x.lower()]
pos_list=[x for x in pos_list if x.split("s/MediaStorm_")[1][:10]>=str(start_date_2019Q4)]
print(len(pos_list))

pos_list.sort()
pos_list

6


['/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-09/MediaStormDailySales20191112-115210-002.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-16/MediaStormDailySales20191119-112232-478.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-23/MediaStormDailySales20191126-112901-552.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-30/MediaStormDailySales20191203.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-07/MediaStormDailySales.txt',
 '/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-14/MediaStormDailySales20191217-195625-000.txt']

In [3]:
pos_list[0]

'/home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-09/MediaStormDailySales20191112-115210-002.txt'

In [5]:
df_all_POS_by_store=pd.DataFrame()
for file in pos_list:
    df=pd.read_csv(file,dtype=str,sep="|")
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df_rewards=df[pd.notnull(df['customer_id_hashed'])]
    df_non_rewards=df[pd.isnull(df['customer_id_hashed'])]
    
    df_rewards_sales=df_rewards.groupby(['location_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"rewards_sales"})
    df_rewards_trans=df_rewards[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_rewards_trans['rewards_trans']=1
    df_rewards_trans=df_rewards_trans.groupby(['location_id','transaction_dt'])['rewards_trans'].sum().to_frame().reset_index()
    df_rewards=pd.merge(df_rewards_sales,df_rewards_trans,on=['location_id','transaction_dt'],how="outer")
    
    df_non_rewards_sales=df_non_rewards.groupby(['location_id','transaction_dt'])['item_transaction_amt'].sum().to_frame().reset_index().rename(columns={"item_transaction_amt":"non_rewards_sales"})
    df_non_rewards_trans=df_non_rewards[['location_id','transaction_dt','transaction_id']].drop_duplicates()
    df_non_rewards_trans['non_rewards_trans']=1
    df_non_rewards_trans=df_non_rewards_trans.groupby(['location_id','transaction_dt'])['non_rewards_trans'].sum().to_frame().reset_index()
    df_non_rewards=pd.merge(df_non_rewards_sales,df_non_rewards_trans,on=['location_id','transaction_dt'],how="outer")
    
    df=pd.merge(df_rewards,df_non_rewards,on=['location_id','transaction_dt'],how="outer")
    df['week_end_dt']=df['transaction_dt'].max()
    df_all_POS_by_store=df_all_POS_by_store.append(df)
    print(datetime.datetime.now(),df['transaction_dt'].max())


2019-12-18 12:00:38.534127 2019-11-09
2019-12-18 12:01:47.298979 2019-11-16
2019-12-18 12:02:49.776680 2019-11-23
2019-12-18 12:04:16.497683 2019-11-30
2019-12-18 12:05:40.352991 2019-12-07
2019-12-18 12:07:12.241777 2019-12-14


In [6]:
df_all_POS_by_store['store_type']=np.where(df_all_POS_by_store['location_id']=="6990","online","instore")
df_summary=df_all_POS_by_store.groupby(["week_end_dt",'store_type'])['rewards_sales','non_rewards_sales','rewards_trans','non_rewards_trans'].sum().reset_index()

In [7]:
writer=pd.ExcelWriter("./BL_first_6_weeks_sales_2019Q4_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_summary.to_excel(writer,"df_summary",index=False)
df_all_POS_by_store.to_excel(writer,"df_by_store_date",index=False)
writer.save()

In [8]:
df_all_POS_by_store.shape

(59641, 8)

In [9]:
df_all_POS_by_store.head(3)

,location_id,transaction_dt,rewards_sales,rewards_trans,non_rewards_sales,non_rewards_trans,week_end_dt,store_type
0,1,2019-11-03,11861.60,213.0,3845.84,154,2019-11-09,instore
1,1,2019-11-04,4648.47,147.0,2840.52,96,2019-11-09,instore
2,1,2019-11-05,4513.96,120.0,2211.68,127,2019-11-09,instore


In [10]:
df_summary

,week_end_dt,store_type,rewards_sales,non_rewards_sales,rewards_trans,non_rewards_trans
0,2019-11-09,instore,6.619332e+07,37157507.01,1657675.0,1301257
1,2019-11-09,online,3.527831e+05,332391.56,3875.0,3773
2,2019-11-16,instore,7.351993e+07,39663995.05,1827533.0,1380695
3,2019-11-16,online,3.587412e+05,365636.26,4247.0,4665
4,2019-11-23,instore,8.563713e+07,46010836.73,1987426.0,1527385
5,2019-11-23,online,4.438943e+05,447768.37,5344.0,5569
6,2019-11-30,instore,1.298683e+08,67029164.93,2605443.0,1975698
7,2019-11-30,online,6.608430e+05,558059.12,8537.0,7415
8,2019-12-07,instore,1.010837e+08,62168641.61,2481072.0,2065753
9,2019-12-07,online,1.058791e+06,863985.40,13668.0,11370


In [11]:
print(datetime.datetime.now())

2019-12-18 12:11:06.918079
